In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from sklearn.metrics import confusion_matrix
import json

# --- CẤU HÌNH BENCHMARK ---
MODEL_FOLDER = "path/to/models/"
PREPARED_FOLDER = "prepared_data/"
RESULTS_DIR = "results/"
BATCH_SIZE = 32
CONTEXT = 5000 # Phải khớp với context lúc prepare data

### Data preparation

In [ ]:
from data_preparation import prepare_csv_datasets
os.makedirs(PREPARED_FOLDER, exist_ok=True)
test_files = ['test_1_1_1.csv', 'test_2_1_1.csv', 'test_4_1_1.csv', 'test_10_1_1.csv', 'test_data.csv']
prepare_csv_datasets(test_files)

### Inference

In [ ]:
# Load tất cả models 1 lần duy nhất bên ngoài vòng lặp file
print("Loading Ensemble Models...")
models = []
for i in range(1, 6):
    model_path = os.path.join(MODEL_FOLDER, f"spliceai{i}.h5")
    models.append(tf.keras.models.load_model(model_path, compile=False))
print(f"Successfully loaded {len(models)} models.")

In [ ]:
from metrics import compute_metrics

os.makedirs(RESULTS_DIR, exist_ok=True)

def one_hot_encode(seq):
    # Vấn đề 4: Xử lý 'N' thành vector 0 (hoặc 0.25 tùy model, ở đây dùng 0)
    map_dict = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    seq_code = np.zeros((len(seq), 4), dtype=np.float32)
    for i, nt in enumerate(seq):
        if nt in map_dict:
            seq_code[i, map_dict[nt]] = 1.0
    return seq_code

def create_dataset(sequences, batch_size):
    # Vấn đề 1: Sử dụng Generator để không phải load toàn bộ mảng X vào RAM
    def gen():
        for s in sequences:
            yield one_hot_encode(s)
            
    return tf.data.Dataset.from_generator(
        gen,
        output_signature=tf.TensorSpec(shape=(2*CONTEXT+1, 4), dtype=tf.float32)
    ).batch(batch_size).prefetch(tf.data.AUTOTUNE)

for csv_file in test_files:
    path = os.path.join(PREPARED_FOLDER, csv_file)
    if not os.path.exists(path): continue
    
    df = pd.read_csv(path)
    y_true = df['Splicing_types'].values
    ds = create_dataset(df['sequence'], BATCH_SIZE)

    # Ensemble Inference
    all_preds = []
    for i, model in enumerate(models):
        print(f"Predicting with model {i+1} for {csv_file}...")
        preds = model.predict(ds, verbose=1)
        # Lấy nucleotide trung tâm (điểm splice)
        all_preds.append(preds[:, CONTEXT, :])

    # Trung bình cộng xác suất
    avg_probs = np.mean(all_preds, axis=0)
    
    # Mapping nhãn theo đúng thứ tự SpliceAI (None:0, Don:1, Acc:2)
    # Lưu ý: SpliceAI nguyên bản trả về [None, Acceptor, Donor]
    # Nếu y_true của bạn là 1:Donor, 2:Acceptor -> Thứ tự: [None, Donor, Acceptor]
    probs_mapped = np.stack([avg_probs[:, 0], avg_probs[:, 2], avg_probs[:, 1]], axis=1)
    preds_mapped = np.argmax(probs_mapped, axis=1)

    # Tính metrics
    results = compute_metrics(y_true, preds_mapped, probs=probs_mapped, k=2)
    cm = confusion_matrix(y_true, preds_mapped)

    # Xuất JSON đúng định dạng yêu cầu
    output = {"metrics": results, "confusion_matrix": cm.tolist()}
    json_path = os.path.join(RESULTS_DIR, csv_file.replace('.csv', '_results.json'))
    
    with open(json_path, 'w') as f:
        json.dump(output, f, indent=4)
    print(f"Results saved to {json_path}")